In [ ]:
from google.colab import files
files.upload()

Saving Dataset2.csv to Dataset2.csv


{'Dataset2.csv': b',id,module,questionText,referenceAnswer,id2,studentAnswer,id3,accuracy,similarity_score,score\r\n0,EM_45b,EM,You used several methods to separate and identify the substances in mock rocks. How did you separate the salt from the water?,"The water was evaporated, leaving the salt.",EM_45b-a1,By letting it sit in a dish for a day.,EM.45b.110.1,incorrect,0.619929732,3\r\n1,EM_45b,EM,You used several methods to separate and identify the substances in mock rocks. How did you separate the salt from the water?,"The water was evaporated, leaving the salt.",EM_45b-a1,Let the water evaporate and the salt is left behind.,EM.45b.113.1,correct,0.979740887,5\r\n2,EM_45b,EM,You used several methods to separate and identify the substances in mock rocks. How did you separate the salt from the water?,"The water was evaporated, leaving the salt.",EM_45b-a1,The water evaporated and left salt crystals.,EM.45b.114.1,correct,0.956945715,5\r\n3,EM_45b,EM,You used several methods to separate 

In [ ]:
import numpy as np
import pandas as pd
import re
import gc
import os
#print(os.listdir("../input"))
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
#tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding
from sklearn.metrics import classification_report
from keras.utils import pad_sequences

ImportError: ignored

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

--2022-11-27 13:56:34--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 172.253.123.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip.1’

uncased_L-12_H-768_ 100%[===================>] 388.84M   221MB/s    in 1.8s    

2022-11-27 13:56:36 (221 MB/s) - ‘uncased_L-12_H-768_A-12.zip.1’ saved [407727028/407727028]

--2022-11-27 13:56:36--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK


In [ ]:
pip install tensorflow==1.15.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import modeling
import optimization
import run_classifier
import tokenization

In [ ]:
import zipfile

In [ ]:
folder = 'model_folder'
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("Dataset2.csv")
df.head()
df=df[:100]

In [ ]:
df.tail()

,Unnamed: 0,id,module,questionText,referenceAnswer,id2,studentAnswer,id3,accuracy,similarity_score,score
94,94,EM_16b,EM,"Ms. Teridann, a geologist, made a chart showin...",Topaz would be the hardest of the 4 minerals b...,EM_16b-a1,Mineral Z can scratch topaz because mineral Z ...,EM.16b.528.1,incorrect,0.831408,4
95,95,EM_16b,EM,"Ms. Teridann, a geologist, made a chart showin...",Topaz would be the hardest of the 4 minerals b...,EM_16b-a1,"It is so strong that minerals X, Y, Z cannot b...",EM.16b.532.1,incorrect,0.553564,2
96,96,EM_16b,EM,"Ms. Teridann, a geologist, made a chart showin...",Topaz would be the hardest of the 4 minerals b...,EM_16b-a1,I know because when you break a rock that is w...,EM.16b.538.1,incorrect,0.539603,2
97,97,EM_16b,EM,"Ms. Teridann, a geologist, made a chart showin...",Topaz would be the hardest of the 4 minerals b...,EM_16b-a1,"How it is compared to X, Y, and Z is that the ...",EM.16b.540.1,incorrect,0.572323,2
98,98,EM_16b,EM,"Ms. Teridann, a geologist, made a chart showin...",Topaz would be the hardest of the 4 minerals b...,EM_16b-a1,I know it is Z because it has one yes and 2 no's.,EM.16b.573.1,incorrect,0.379464,1


In [ ]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/outputs'
print(f'>> Model output directory: {OUTPUT_DIR}')
print(f'>> BERT pretrained directory: {BERT_PRETRAINED_DIR}')

>> Model output directory: model_folder/outputs
>> BERT pretrained directory: model_folder/uncased_L-12_H-768_A-12


In [ ]:
sentences1 = df.questionText.values
sentences2 = df.studentAnswer.values
sentencesf=[]
sentences = [sentence for sentence in sentences1]
#sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences1]
for i in range(len(sentences)):
  sent=sentences[i] +  " [SEP] "+ sentences2[i]
  sentencesf.append(sent)
sentencesf


['You used several methods to separate and identify the substances in mock rocks. How did you separate the salt from the water? [SEP] By letting it sit in a dish for a day.',
 'You used several methods to separate and identify the substances in mock rocks. How did you separate the salt from the water? [SEP] Let the water evaporate and the salt is left behind.',
 'You used several methods to separate and identify the substances in mock rocks. How did you separate the salt from the water? [SEP] The water evaporated and left salt crystals.',
 'You used several methods to separate and identify the substances in mock rocks. How did you separate the salt from the water? [SEP] I saw a pinkish grayish color that was blocking the water.',
 'You used several methods to separate and identify the substances in mock rocks. How did you separate the salt from the water? [SEP] You have to slowly tip the vial for only the water to go.',
 'You used several methods to separate and identify the substances

In [ ]:
df2 = pd.DataFrame()
df2["Text"] = sentencesf
df2["Label"] = df['score']
df2=df2[:200]

df2.head()

,Text,Label
0,You used several methods to separate and ident...,3
1,You used several methods to separate and ident...,5
2,You used several methods to separate and ident...,5
3,You used several methods to separate and ident...,2
4,You used several methods to separate and ident...,3


In [ ]:
df.tail()

,Unnamed: 0,id,module,questionText,referenceAnswer,id2,studentAnswer,id3,accuracy,similarity_score,score
94,94,EM_16b,EM,"Ms. Teridann, a geologist, made a chart showin...",Topaz would be the hardest of the 4 minerals b...,EM_16b-a1,Mineral Z can scratch topaz because mineral Z ...,EM.16b.528.1,incorrect,0.831408,4
95,95,EM_16b,EM,"Ms. Teridann, a geologist, made a chart showin...",Topaz would be the hardest of the 4 minerals b...,EM_16b-a1,"It is so strong that minerals X, Y, Z cannot b...",EM.16b.532.1,incorrect,0.553564,2
96,96,EM_16b,EM,"Ms. Teridann, a geologist, made a chart showin...",Topaz would be the hardest of the 4 minerals b...,EM_16b-a1,I know because when you break a rock that is w...,EM.16b.538.1,incorrect,0.539603,2
97,97,EM_16b,EM,"Ms. Teridann, a geologist, made a chart showin...",Topaz would be the hardest of the 4 minerals b...,EM_16b-a1,"How it is compared to X, Y, and Z is that the ...",EM.16b.540.1,incorrect,0.572323,2
98,98,EM_16b,EM,"Ms. Teridann, a geologist, made a chart showin...",Topaz would be the hardest of the 4 minerals b...,EM_16b-a1,I know it is Z because it has one yes and 2 no's.,EM.16b.573.1,incorrect,0.379464,1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2["Text"].values, df2["Label"].values, test_size=0.1, random_state=2)

In [ ]:
import os

In [ ]:
import tensorflow as tf

In [ ]:
def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 2
EVAL_BATCH_SIZE = 2
LEARNING_RATE =0.7
NUM_TRAIN_EPOCHS = 1
WARMUP_PROPORTION = 1
MAX_SEQ_LENGTH = 10
# Model configs
SAVE_CHECKPOINTS_STEPS = 0 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 1
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(8)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
import datetime

In [ ]:
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.




Please wait...
>> Started training at 2022-11-27 13:57:17.445277 
  Num examples = 89
  Batch size = 2



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


>> Finished training at 2022-11-27 13:59:32.620849


In [ ]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)
  def input_fn(params):
    print(params)
    batch_size = 500
    num_examples = len(features)
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })
    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)
    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d
  return input_fn

In [ ]:
pip install numpy==1.19.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
predict_examples = create_examples(X_test, 'test')
predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)
result = estimator.predict(input_fn=predict_input_fn)
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))

{}


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy of BERT is:",accuracy_score(y_test,preds))
print(classification_report(y_test,preds))

Accuracy of BERT is: 0.4
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           5       0.44      1.00      0.62         4

    accuracy                           0.40        10
   macro avg       0.11      0.25      0.15        10
weighted avg       0.18      0.40      0.25        10



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
l=['You used several methods to separate and identify the substances in mock rocks. How did you separate the salt from the water? [SEP] Let the water evaporate and the salt is left behind.']
def result(l):
  predict_examples = create_examples(l, 'test')
  predict_features = run_classifier.convert_examples_to_features(
      predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = input_fn_builder(
      features=predict_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=False)
  result = estimator.predict(input_fn=predict_input_fn)
  preds = []
  for prediction in result:
        preds.append(np.argmax(prediction['probabilities']))
  return preds

In [ ]:
def Question():
  df1=pd.read_csv("Dataset2.csv")
  import random
  i=random.randrange(0,100)
  print("Question=",df1['questionText'][i])
  print("Refernce Answer=",df1['referenceAnswer'][i])
  Answer=input('Enter The Answer=')
  l=[]
  s=df1['questionText'][i] +" [SEP] "+ Answer
  l.append(s)
  r=result(l)
  return r


In [ ]:
r= Question()
print("Your Score is ",r)

Question= You used several methods to separate and identify the substances in mock rocks. How did you separate the salt from the water?
Refernce Answer= The water was evaporated, leaving the salt.
Enter The Answer=Evaporation
{}
Your Score is  [4]


In [ ]:
r= Question()
print("Your Score is ",r)

Question= Ms. Teridann, a geologist, made a chart showing the scratch-test data for 3 minerals (Mineral X, Mineral Y, and Mineral Z). She wrote "yes" if the tool scratched the mineral and wrote "no" if it did not scratch it. Topaz is a mineral that cannot be scratched by a fingernail, a penny, or a paperclip. How hard is topaz compared to minerals X, Y, and Z? Explain how you know.
Refernce Answer= Topaz would be the hardest of the 4 minerals because none of the tools could scratch topaz and at least one tool could scratch each of the other minerals.
Enter The Answer=yes
{}
Your Score is  [7]


In [ ]:
Question()
print("Your Score is ",r)
edit=input('Do you Want to edit the Score(Y/N)=')
if edit=='Y' or edit=='y':
  r=input('Enter the new Score=')

print("Your Score is ",r)

Question= You used several methods to separate and identify the substances in mock rocks. How did you know the crystals were salt?
Refernce Answer= The crystals were square with Xs on the surface.
Enter The Answer=yes
{}
Your Score is  [4]
Do you Want to edit the Score(Y/N)=yes
Your Score is  [4]
